In [1]:
%matplotlib inline
import numpy as np
import os, string
from matplotlib import pyplot as plt
import tensorflow as tf
import seaborn as sns

states = ('Rainy', 'Sunny')
 
observations = ('walk', 'shop', 'clean')
 
start_probability = {'Rainy': 0.6, 'Sunny': 0.4}
 
transition_probability = {
    'Rainy' : {'Rainy': 0.7, 'Sunny': 0.3},
    'Sunny' : {'Rainy': 0.4, 'Sunny': 0.6},
    }
 
emission_probability = {
    'Rainy' : {'walk': 0.1, 'shop': 0.4, 'clean': 0.5},
    'Sunny' : {'walk': 0.6, 'shop': 0.3, 'clean': 0.1},
}

stateNum = 2
estateNum = 3
p0=np.array([0.6,0.4])
T = np.array([[0.7,0.3],[0.4,0.6]])
E = np.array([[0.1,0.4,0.5],[0.6,0.3,0.1]])
obchain = np.array([0,1,2])
print(p0)
print(T)
print(E)

[ 0.6  0.4]
[[ 0.7  0.3]
 [ 0.4  0.6]]
[[ 0.1  0.4  0.5]
 [ 0.6  0.3  0.1]]


In [94]:
with tf.device('/cpu:0'):
    start = tf.placeholder('float',p0.shape)
    tT = tf.placeholder('float',T.shape) 
    tE = tf.placeholder('float',E.shape)
    y = tf.placeholder('float',[obchain.shape[0],estateNum])
    state = tf.Variable(tf.random_uniform(p0.shape,0.,1.0))
    state.assign(start)
    latentChainProb=[]
    chainProb = []
    for i in range(0,obchain.shape[0]):
        latentChainProb.append(tf.Variable(tf.random_uniform([1,2],0.,1.0)))
        
    for i in range(0,obchain.shape[0]):
        latentChainProb[i] = tf.div(latentChainProb[i],tf.reduce_sum(latentChainProb[i]))
        state = tf.mul(state,latentChainProb[i])
        #print(state.get_shape(), tT.get_shape())
        state = tf.minimum(tf.maximum(tf.matmul(state,tT),0.0),1.)
        estate = tf.minimum(tf.maximum(tf.matmul(state,tE),0.),1.)
        #estate = tf.cast(tf.argmax(estate,dimension=1),tf.float32)
        #print(estate.get_shape())
        estate = tf.div(estate,tf.reduce_sum(estate))
        chainProb.append(estate)
    
    epredict = tf.concat(0,chainProb)
    print(epredict.get_shape())
    loss = tf.nn.l2_loss(epredict-y)
    train_op = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
    
init = tf.global_variables_initializer ()
# Launch the graph.
sess = tf.Session()
sess.run(init)

(3, 3)


In [95]:
import sys,time
y_=np.zeros([obchain.shape[0],estateNum],np.int)
for i in range(0,obchain.shape[0]):
    y_[obchain[i],i] = 1
print(y_)
for i in range(100):
    #train
    _,epre,curloss = sess.run([train_op,epredict,loss], feed_dict={start: p0, y: y_, tT: T, tE:E})
    if i%20 == 0:
        print(i,epre,curloss)
        #sys.stdout.write('\r epoch : %d' % i)
        #sys.stdout.flush()
        #time.sleep(0.2)
        

[[1 0 0]
 [0 1 0]
 [0 0 1]]
0 [[ 0.36770934  0.34645814  0.28583252]
 [ 0.35872072  0.34825587  0.29302347]
 [ 0.29234347  0.36153132  0.34612522]] 0.94228
20 [[ 0.60000002  0.30000001  0.1       ]
 [ 0.40000001  0.33999997  0.25999996]
 [ 0.1         0.40000001  0.5       ]] 0.6716
40 [[ 0.60000002  0.30000001  0.1       ]
 [ 0.40000001  0.33999997  0.25999996]
 [ 0.1         0.40000001  0.5       ]] 0.6716
60 [[ 0.60000002  0.30000001  0.1       ]
 [ 0.40000001  0.33999997  0.25999996]
 [ 0.1         0.40000001  0.5       ]] 0.6716
80 [[ 0.60000002  0.30000001  0.1       ]
 [ 0.40000001  0.33999997  0.25999996]
 [ 0.1         0.40000001  0.5       ]] 0.6716


In [96]:
print(sess.run(latentChainProb[0]))
print(sess.run(latentChainProb[1]))
print(sess.run(latentChainProb[2]))

[[-1.558689    2.55868888]]
[[ 0.36379731  0.63620269]]
[[ 2.13955188 -1.13955188]]


In [81]:
help(tf.argmax)

Help on function argmax in module tensorflow.python.ops.math_ops:

argmax(input, axis=None, name=None, dimension=None)
    Returns the index with the largest value across axiss of a tensor.
    
    Args:
      input: A `Tensor`. Must be one of the following types: `float32`, `float64`, `int64`, `int32`, `uint8`, `uint16`, `int16`, `int8`, `complex64`, `complex128`, `qint8`, `quint8`, `qint32`, `half`.
      axis: A `Tensor`. Must be one of the following types: `int32`, `int64`.
        int32, 0 <= axis < rank(input).  Describes which axis
        of the input Tensor to reduce across. For vectors, use axis = 0.
      name: A name for the operation (optional).
    
    Returns:
      A `Tensor` of type `int64`.



In [5]:
a=np.matmul(np.multiply(p0,T),E)
print(a)
print(a.sum(axis=0))

[[ 0.114  0.204  0.222]
 [ 0.168  0.168  0.144]]
[ 0.282  0.372  0.366]


In [6]:
print(0.282 + 0.372 + 0.366)

1.02
